## Figure 5
Difference in CDNC between 3 surface-partitioning models and constant surface tension at +200m in parcel simulation. 
Columns show results from different models (OV, SL, RU).
Rows show slices through the 3-dimensional phase space of Na-w-Forg.

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install atmos-cloud-sim-uj-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [2]:
import numpy as np
import os
from joblib import Parallel, delayed
import pickle as pkl

from matplotlib import pyplot
from matplotlib.colors import TwoSlopeNorm
from open_atmos_jupyter_utils import show_plot

from PySDM import Formulae
from PySDM.physics import si
from PySDM.initialisation.sampling import spectral_sampling as spec_sampling

from PySDM_examples.Singer_Ward import Settings, ParcelSimulation
from PySDM_examples.Singer_Ward.aerosol import AerosolBetaCaryophylleneDark
from PySDM_examples.Singer_Ward.constants_def import SINGER_CONSTS, plot_names

import warnings
from numba.core.errors import NumbaExperimentalFeatureWarning

In [3]:
CI = 'CI' in os.environ
n = 4

save = True
restore_saved_data = False

updrafts = np.geomspace(0.1, 10, 2 if CI else n)
forgs = np.linspace(0.05, 0.95, 2 if CI else n)
aerosol_number_concs = np.geomspace(50, 400, 2 if CI else n)
models = ('Constant', 'CompressedFilmOvadnevaite', 'SzyszkowskiLangmuir', 'CompressedFilmRuehl')

FORMULAE = Formulae(constants=SINGER_CONSTS)
# WATER_MOLAR_VOLUME = FORMULAE.constants.water_molar_volume
WATER_MOLAR_VOLUME = FORMULAE.constants.Mv / FORMULAE.constants.rho_w

In [4]:
def genkey(w, Forg, N_a, model):
    return f"w{w:.2f}_f{Forg:.2f}_N{N_a:.2f}_{model}"

if restore_saved_data:
    with open('data_fig5.pkl','rb') as file:
        output = pkl.load(file)
else:
    def compute(key, settings):
        simulation = ParcelSimulation(settings)
        output = simulation.run()
        output['updraft'] = settings.w
        output['org_fraction'] = settings.aerosol.modes[0]['f_org']
        output['color'] = settings.aerosol.color
        return key, output

    print(f'tasks scheduled: {len(models) * len(forgs) * len(updrafts) * len(aerosol_number_concs)}')
    output = dict(Parallel(verbose=10, n_jobs=-1)(
        delayed(compute)(genkey(w, Forg, N_a, model), Settings(
            dz = 1 * si.m, 
            n_sd_per_mode = 100, 
            model = model,
            aerosol = AerosolBetaCaryophylleneDark(water_molar_volume=WATER_MOLAR_VOLUME, Forg=Forg, N_a=N_a, r_a=50),
            w = w * si.m / si.s,
            spectral_sampling = spec_sampling.ConstantMultiplicity,
        ))
        for w in updrafts
        for Forg in forgs
        for N_a in aerosol_number_concs
        for model in models
    ))

    if save:
        with open('data_fig5.pkl','wb') as file:
            pkl.dump(output, file)

tasks scheduled: 256


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.4min
TOMS748 problem: not fa * fb < 0
	file: /Users/claresinger/Research/PySDM/PySDM/backends/impl_numba/toms748.py
TOMS748 problem: not fa * fb < 0
	file: /Users/claresinger/Research/PySDM/PySDM/backends/impl_numba/toms748.py
TOMS748 problem: not fa * fb < 0
	file: /Users/claresinger/Research/PySDM/PySDM/backends/impl_numba/toms748.py
TOMS748 problem: not fa * fb < 0
	file: /Users/claresinger/Research/PySDM/PySDM/backends/impl_numba/toms748.py
TOMS748 problem: not fa * fb < 0
	file: /Users/claresinger/Research/PySDM/PySDM/backends/impl_numba/toms748.py
TOMS748 problem: not fa * fb < 0
	file: /Users/claresinger/Research/PySDM/PySDM/backends/impl_numba/toms748.py
TOMS748 problem: not fa * fb < 0
	file: /Users/claresinger/Research/PySDM/PySDM/backends/impl_numba/toms748.py
TOMS748 problem: not fa

AssertionError: 
Items are not equal to 5 significant digits:
 ACTUAL: 99998000.00000001
 DESIRED: 99999999.99999999

In [ ]:
dCDNC = np.zeros((len(models), len(updrafts), len(forgs), len(aerosol_number_concs)))
for m in range(0,3):
    for i,w in enumerate(updrafts):
        for j,Forg in enumerate(forgs):
            for k,Na in enumerate(aerosol_number_concs):
                key_film = genkey(w, Forg, Na, models[m+1])
                key_bulk = genkey(w, Forg, Na, "Constant")
                z = np.array(output[key_bulk]['z'])
                wz = np.where(z == z[-1])[0][0]
                CDNC_film = np.array(output[key_film]['CDNC'])[wz]
                CDNC_bulk = np.array(output[key_bulk]['CDNC'])[wz]
                dCDNC[m,i,j,k] = (CDNC_film - CDNC_bulk) / CDNC_bulk * 100.0

pyplot.rcParams.update({"font.size":10})
fig, axes = pyplot.subplots(3, 4, figsize=(10,10), gridspec_kw={"width_ratios":[1,1,1,0.05]}, layout="constrained")

x = {0:forgs, 1:aerosol_number_concs, 2:aerosol_number_concs}
xlab = {0: "$F_{org}$", 1:"$N_a$ [cm$^{-3}$]", 2:"$N_a$ [cm$^{-3}$]"}
xscale = {0: "linear", 1:"log", 2:"log"}
xtickloc = {0:[0.25, 0.5, 0.75], 1:[50, 100, 200, 400], 2:[50, 100, 200, 400]}
xticklab = {0:["0.25", "0.5", "0.75"], 1:["50","100","200","400"], 2:["50","100","200","400"]}

y = {0:updrafts, 1:updrafts, 2:forgs}
ylab = {0: "$w$ [m s$^{-1}$]", 1: "$w$ [m s$^{-1}$]", 2:"$F_{org}$"}
yscale = {0: "log", 1: "log", 2:"linear"}
ytickloc = {0:[0.1,1,10], 1:[0.1,1,10], 2:[0.25, 0.5, 0.75]}
yticklab = {0:["0.1","1","10"], 1:["0.1","1","10"], 2:["0.25", "0.5", "0.75"]}

for row in range(0,3):
    for i in range(0,3):
        ax = axes[row, i]
        if row == 0:
            ax.set_title(plot_names[models[i+1]], fontweight="bold")
        
        levs = np.linspace(-50,50,40)
        if row == 0:
            z = dCDNC[i,:,:,1]
        if row == 1:
            z = dCDNC[i,:,1,:]
        if row == 2:
            z = dCDNC[i,1,:,:]
        
        p = ax.contourf(x[row], y[row], z, 
            cmap="bwr", levels=levs, extend="both")
        
        ax.set_xlabel(xlab[row])
        ax.set_ylabel(ylab[row])
        ax.set_xscale(xscale[row])
        ax.set_yscale(yscale[row])
        ax.set_xticks(xtickloc[row], labels=xticklab[row])
        ax.set_yticks(ytickloc[row], labels=yticklab[row])

        pyplot.colorbar(p, cax=axes[row,-1], label=r"$\Delta$CDNC [%]")

show_plot("fig5.pdf")